## IMPORTS
No need to change

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import time, math, re

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chrome_options=chrome_options)

## MANUAL NAVIGATION
The chrome browser must have opened, in the needs go to the first page of tutors with results 1-10 and copy the url, paste it in place of URL below

In [50]:
URL = "https://www.urbanpro.com/need/applicantListForSeeker?id=7135438&srtBy=URBANPRORATING&af=0&page=1&tc=25385&onlnCls=on&hmTn=false&rglrCls=false&pincode=226012%20(AASHIYANA,%20Lucknow)&ATTR1=178&ATTR4=English&ATTR4=Sanskrit&ATTR4=Telugu&ATTR4=Gujarati&ATTR4=Mathematics&ATTR4=Science&ATTR4=Oriya&ATTR4=Marathi&ATTR4=Tamil&ATTR4=Assamese&ATTR4=Punjabi&ATTR4=Japanese&ATTR4=French&ATTR4=Bengali&ATTR4=Social%20Science&ATTR4=Kannada&ATTR4=EVS&ATTR4=Hindi&ATTR4=German&ATTR4=Computers&ATTR4=Urdu&ATTR4=Spanish&ATTR4=Malayalam&ATTR4=Manipuri"
PAGES = math.ceil((int(re.findall("tc=[0-9]+",URL)[0][3:]))/10)
URL=URL.replace("page=1", "page={}")
TUTORS=[]
f=open("log_urls.txt","w+")

In [51]:
def extract(sleep=30):
    _ = WebDriverWait(driver, sleep).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='providerCard bodyTxt']")))
    page = bs(driver.page_source)
    refs = page.find_all('a',{'class':'gaClickEventTrack'})
    links = ['https://www.urbanpro.com'+link.attrs['href'] for link in refs]
    TUTORS.extend(links)
    f.write('\n'.join(TUTORS)+'\n')
    print("{} Tutors are scraped".format(len(TUTORS)))

## Uncomment the ITER=PAGES line below for actual scraping

In [49]:
ITER = 10
#ITER = PAGES

for i in range(ITER):
    driver.get(URL.format(i+1))
    extract()
f.close()

10 Tutors are scraped


In [22]:
f=open("log_profiles",'w+')

In [37]:
def get_profile(url, sleep=10):
    driver.get(url)
    contact = WebDriverWait(driver, sleep).until(EC.visibility_of_element_located((By.XPATH, "//a[@class='primaryButton viewNumber messageCallFlow viewNumberMob']")))
    page = bs(driver.page_source)
    contact.click()
    contact = WebDriverWait(driver, sleep).until(EC.visibility_of_element_located((By.XPATH, "//a[@class='providerExotelPhoneNumber']")))
    out = (page.find('div',{'class':'profileContentContainer'}), page.find('div',{'class':'profileContentContainer overviewClassesTab'}), contact.text)
    f.write(str(out)+'\n')
    return out

In [40]:
profiles = [get_profile(tutor_link) for tutor_link in TUTORS